In [1]:
%load_ext autoreload
%autoreload 2

In [69]:
import numpy as np
import polars as pl
from pyxirr import irr, npv
from functools import partial
from scipy.optimize import fsolve

In [44]:

from schema import SolarPVAssumptions
# Example usage
assumptions = SolarPVAssumptions(
    capacity_mw=30,
    capital_expenditure_per_mw=670_000,
    o_m_cost_pct_of_capital_cost=0.02,
    capacity_factor=0.097,
    project_lifetime_years=20,
    debt_pct_of_capital_cost=0.871,
    equity_pct_of_capital_cost=0.129,
    cost_of_debt=0.04,
    cost_of_equity=0.12,
    tax_rate=0.25,
    dcsr=1.3
)

print(assumptions)

capacity_mw=30.0 capacity_factor=0.097 capital_expenditure_per_mw=670000.0 o_m_cost_pct_of_capital_cost=0.02 debt_pct_of_capital_cost=0.871 equity_pct_of_capital_cost=0.129 cost_of_debt=0.04 cost_of_equity=0.12 tax_rate=0.25 project_lifetime_years=20 dcsr=1.3 capital_cost=20100000.0 tax_adjusted_WACC=0.04161 wacc=0.050320000000000004


In [ ]:
## Pro Forma

# Starting LCOE
LCOE_guess = 81.44

def calculate_cashflow(assumptions, LCOE_guess, return_model=False):
    # Create a dataframe, starting with the period
    model = pl.DataFrame(
        {
            "Period": [i for i in range(assumptions.project_lifetime_years + 1)],
        }
    )

    model = model.with_columns(
        Capacity_MW = pl.when(pl.col("Period") > 0).then(assumptions.capacity_mw).otherwise(0),
        Capacity_Factor = pl.when(pl.col("Period") > 0).then(assumptions.capacity_factor).otherwise(0),
        LCOE = pl.when(pl.col("Period") > 0).then(LCOE_guess).otherwise(0),
    ).with_columns(
        Total_Generation_MWh = pl.col("Capacity_MW") * pl.col("Capacity_Factor") * 8760,
    ).with_columns(
        Total_Revenues_mn = pl.col("Total_Generation_MWh") * pl.col("LCOE")/1000,
        O_M_Costs_mn = pl.when(pl.col("Period") > 0).then(assumptions.capital_cost/1000 * assumptions.o_m_cost_pct_of_capital_cost).otherwise(0),
    ).with_columns(
        Total_Operating_Costs_mn = pl.col("O_M_Costs_mn"),
    ).with_columns(
        EBITDA_mn = pl.col("Total_Revenues_mn") - pl.col("Total_Operating_Costs_mn"),
    ).with_columns(
        CFADS_mn = pl.col("EBITDA_mn"),
    ).with_columns(
        Target_Debt_Service_mn = pl.when(pl.col("Period") == 0).then(0).otherwise(pl.col("CFADS_mn")/assumptions.dcsr),
        Debt_Outstanding_EoP_mn = pl.when(pl.col("Period") == 0).then(assumptions.debt_pct_of_capital_cost * assumptions.capital_cost/1000).otherwise(0),
    ).with_columns(
        Interest_Expense_mn = pl.when(pl.col("Period") == 0).then(0).otherwise(pl.col("Debt_Outstanding_EoP_mn").shift(1) * assumptions.cost_of_debt),
    ).with_columns(
        Amortization_mn = pl.when(pl.col("Period") == 0).then(0).otherwise(pl.min_horizontal(pl.col("Target_Debt_Service_mn") - pl.col("Interest_Expense_mn"), pl.col("Debt_Outstanding_EoP_mn").shift(1))),
    ).with_columns(
        Debt_Outstanding_EoP_mn = pl.when(pl.col("Period") == 0).then(pl.col(
                "Debt_Outstanding_EoP_mn"
        )).otherwise(pl.col("Debt_Outstanding_EoP_mn").shift(1) - pl.col("Amortization_mn")
    )).with_columns(
        Debt_Outstanding_BoP_mn = pl.col("Debt_Outstanding_EoP_mn").shift(1),
    ).to_pandas()

    for period in model["Period"]:
        if period > 1:
            model.loc[period, "Interest_Expense_mn"] = model.loc[period, "Debt_Outstanding_BoP_mn"] * assumptions.cost_of_debt
            model.loc[period, "Amortization_mn"] = min(
                model.loc[period, "Target_Debt_Service_mn"] - model.loc[period, "Interest_Expense_mn"],
                model.loc[period, "Debt_Outstanding_BoP_mn"]
            )
            model.loc[period, "Debt_Outstanding_EoP_mn"] = model.loc[period, "Debt_Outstanding_BoP_mn"] - model.loc[period, "Amortization_mn"]
            if period < assumptions.project_lifetime_years:
                model.loc[period + 1, "Debt_Outstanding_BoP_mn"] = model.loc[period, "Debt_Outstanding_EoP_mn"]

    model = pl.DataFrame(model).with_columns(
        # Straight line depreciation
        Depreciation_mn = pl.when(pl.col("Period") > 0).then(assumptions.capital_cost/1000/assumptions.project_lifetime_years).otherwise(0),
    ).with_columns(
        Taxable_Income_mn = pl.col("EBITDA_mn") - pl.col("Depreciation_mn") - pl.col("Interest_Expense_mn"),
    ).with_columns(
        Tax_Liability_mn = pl.max_horizontal(0, assumptions.tax_rate * pl.col("Taxable_Income_mn"))
    ).with_columns(
        Post_Tax_Net_Equity_Cashflow_mn = pl.when(pl.col("Period") == 0).then(-assumptions.capital_cost/1000 * assumptions.equity_pct_of_capital_cost).otherwise(pl.col("EBITDA_mn") - pl.col("Target_Debt_Service_mn") - pl.col("Tax_Liability_mn"))
    )

    # Calculate Post-Tax Equity IRR
    post_tax_equity_irr = irr(model["Post_Tax_Net_Equity_Cashflow_mn"].to_numpy())
    if return_model:
        return model, post_tax_equity_irr
    return post_tax_equity_irr - assumptions.cost_of_equity

In [99]:
breakeven_LCOE = fsolve(partial(calculate_cashflow, assumptions), 44)[0] + 0.0001
calculate_cashflow(assumptions, breakeven_LCOE, return_model=True)


(shape: (21, 19)
 ┌────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
 │ Period ┆ Capacity_M ┆ Capacity_F ┆ LCOE      ┆ … ┆ Depreciat ┆ Taxable_I ┆ Tax_Liabi ┆ Post_Tax_ │
 │ ---    ┆ W          ┆ actor      ┆ ---       ┆   ┆ ion_mn    ┆ ncome_mn  ┆ lity_mn   ┆ Net_Equit │
 │ i64    ┆ ---        ┆ ---        ┆ f64       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ y_Cashflo │
 │        ┆ f64        ┆ f64        ┆           ┆   ┆ f64       ┆ f64       ┆ f64       ┆ w_m…      │
 │        ┆            ┆            ┆           ┆   ┆           ┆           ┆           ┆ ---       │
 │        ┆            ┆            ┆           ┆   ┆           ┆           ┆           ┆ f64       │
 ╞════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
 │ 0      ┆ 0.0        ┆ 0.0        ┆ 0.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ -2592.9   │
 │ 1      ┆ 30.0       ┆ 0.097      ┆ 77.609918 ┆ … ┆ 1005.0    ┆